#### smote + enn
          precision    recall  f1-score   support

          0       0.98      0.96      0.97      1920
          1       0.35      0.51      0.41        80

avg / total       0.95      0.94      0.95      2000

ACCURACY: 0.942

#### smote enn 결측치 2개 delete 했을 때

          precision    recall  f1-score   support

          0       0.98      0.95      0.96      1920
          1       0.28      0.51      0.37        80

avg / total       0.95      0.93      0.94      2000

ACCURACY: 0.929

#### ADASYN

          precision    recall  f1-score   support

          0       0.98      0.97      0.97      1920
          1       0.38      0.46      0.42        80

avg / total       0.95      0.95      0.95      2000

ACCURACY: 0.9485


#### Random oversampling
             precision    recall  f1-score   support

          0       0.98      0.94      0.96      1920
          1       0.30      0.65      0.41        80

avg / total       0.96      0.93      0.94      2000

ACCURACY: 0.9265


### 비식별처리 replace , fillna 해도 비식별처리로 인식됨

In [21]:
import pandas as pd
import numpy as np
import os
from imblearn.combine import SMOTEENN
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn import linear_model
from sklearn.metrics import classification_report
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler ,normalize
from sklearn import preprocessing
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylabA
import seaborn as sns



In [22]:
df = pd.read_csv('./Data_set.csv',encoding='cp949')
df1 = pd.read_csv('./Test_set.csv',encoding='cp949')
# df1.AGE.value_counts(0)

In [23]:
df.AGE

df.AGE = df.AGE.replace('*',46)
df.AGE = pd.Series(df.AGE).convert_objects(convert_numeric=True)
df.AGE = df.AGE.replace(0,df.AGE.mean())
# df.AGE


/home/rosa/miniconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  after removing the cwd from sys.path.


In [24]:
df.SEX
#다음엔 성별을 *로 유지해보기
# df.SEX = df.SEX.replace('*',1)
df.SEX = pd.Series(df.SEX).convert_objects(convert_numeric=True)


/home/rosa/miniconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  This is separate from the ipykernel package so we can avoid doing imports until


In [25]:
df.TEL_CNTT_QTR = df.TEL_CNTT_QTR.astype(str)
df.MIN_CNTT_DATE = df.MIN_CNTT_DATE.astype(str)

In [26]:
#null처리
# df.PAYM_METD = df.PAYM_METD.fillna('O')
df.TEL_MBSP_GRAD = df.TEL_MBSP_GRAD.fillna('R')
df.MATE_OCCP_NAME_G = df.MATE_OCCP_NAME_G.fillna('주부')
df.OCCP_NAME_G = df.OCCP_NAME_G.fillna('주부')
df.LAST_CHLD_AGE = df.LAST_CHLD_AGE.fillna(df.LAST_CHLD_AGE.mean())

In [27]:
#무조건
del df['CUST_ID']
# null
# del df['MATE_OCCP_NAME_G']
# del df['OCCP_NAME_G']
# del df['LAST_CHLD_AGE']
# del df['TEL_MBSP_GRAD']
# del df['PAYM_METD']
del df['PAYM_METD']
del df['STRT_CRDT_GRAD']
del df['FYCM_PAID_AMT']
del df['FMLY_PLPY_CNT']
del df['FMLY_CLAM_CNT']


In [28]:
df['total_cnt'] = df['SPART_LNIF_CNT']+df['ECT_LNIF_CNT']+df['BNK_LNIF_CNT']+df['CPT_LNIF_CNT']
df['total_amt'] = df['TOT_LNIF_AMT']+df['TOT_CLIF_AMT']+df['BNK_LNIF_AMT']+df['CPT_LNIF_AMT']
df['total_occr_mdif'] =df['CRDT_OCCR_CNT']+ df['CTCD_OCCR_MDIF']

In [29]:
del df['BNK_LNIF_CNT']
del df['CPT_LNIF_CNT']
del df['SPART_LNIF_CNT']
del df['ECT_LNIF_CNT']

del df['TOT_LNIF_AMT']
del df['TOT_CLIF_AMT']
del df['BNK_LNIF_AMT']
del df['CPT_LNIF_AMT']

del df['CRDT_CARD_CNT']
del df['CTCD_OCCR_MDIF']

In [33]:
df = pd.get_dummies(df)


In [34]:
df_test = df[:2000]
df_train = df[2000:]

In [35]:
x_data = df_train.drop('TARGET',axis=1)
y_data = df_train.TARGET

In [36]:
X = x_data
y = y_data

In [37]:
df.corrwith(y_data).sort_values(ascending=True)

total_occr_mdif             -0.178275
total_amt                   -0.074545
LINE_STUS_U                 -0.073696
LT1Y_PEOD_RATE_0            -0.065758
CBPT_MBSP_YN_Y              -0.047461
MOBL_FATY_PRC               -0.043736
MOBL_PRIN                   -0.041667
AGE                         -0.036190
LAST_CHLD_AGE               -0.022036
TEL_MBSP_GRAD_E             -0.016049
TEL_MBSP_GRAD_R             -0.013024
CUST_FMLY_NUM               -0.012149
HSHD_INFR_INCM              -0.011874
AVG_CALL_FREQ               -0.011864
TEL_CNTT_QTR_20161          -0.009623
ACTL_FMLY_NUM               -0.008925
TEL_CNTT_QTR_20143          -0.008758
TEL_CNTT_QTR_20151          -0.008507
OCCP_NAME_G_공무원             -0.008167
TEL_CNTT_QTR_20074          -0.007881
TEL_CNTT_QTR_20023          -0.007418
MATE_OCCP_NAME_G_기타         -0.007298
MATE_OCCP_NAME_G_1차산업 종사자   -0.006627
TEL_CNTT_QTR_20162          -0.005991
TEL_CNTT_QTR_20021          -0.005767
TEL_CNTT_QTR_20001          -0.005702
TEL_CNTT_QTR

In [39]:
# df['PAYM_METD_G']*=2
df['LT1Y_MXOD_AMT']*=2
df['CRMM_OVDU_AMT']*=2
df['total_occr_mdif']*=2
df['SPTCT_OCCR_MDIF']*=2
df['total_cnt']*=2

In [40]:
# Apply SMOTE + ENN
sm = SMOTEENN()
X_resampled, y_resampled = sm.fit_sample(X, y)

In [41]:
df_1 = pd.DataFrame(X_resampled)
df_2 = pd.DataFrame(y_resampled)
df_2.columns = ['TARGET']
df_final = pd.concat([df_1,df_2],axis=1)
df_final.head()

,0,1,2,3,4,5,6,7,8,9,...,381,382,383,384,385,386,387,388,389,TARGET
0,1.0,1.0,0.0,0.0,4700.0,8800.0,4.0,1.0,19.0,4200.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
1,25.0,0.0,0.0,0.0,6000.0,7000.0,4.0,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
2,1.0,0.0,0.0,0.0,0.0,8900.0,2.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
3,1.0,0.0,0.0,0.0,5000.0,11700.0,3.0,1.0,19.0,6800.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
4,37.0,0.0,0.0,0.0,5800.0,10500.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0


In [42]:
df_final = df_final.sample(frac=1)
df_final.head()

,0,1,2,3,4,5,6,7,8,9,...,381,382,383,384,385,386,387,388,389,TARGET
28500,0.000000,0.000000,0.000000,0.000000,0.000000,8300.000000,4.000000,2.0,24.000000,5400.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,1.0,0
107117,18.648382,17.415845,0.529301,20643.286755,2259.352929,8199.437513,2.587904,1.0,12.703235,2858.227955,...,0.0,0.0,0.0,0.0,0.0,0.470699,0.0,0.0,1.0,1
10130,97.000000,0.000000,0.000000,0.000000,0.000000,5500.000000,3.000000,2.0,44.000000,5200.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,1.0,0
74813,49.000000,61.000000,0.000000,0.000000,3300.000000,3300.000000,4.000000,1.0,24.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,1.0,1
111433,1.000000,37.000000,0.012277,0.012277,0.000000,104.356242,1.012277,1.0,0.049109,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,1.0,1


In [43]:
train_y = df_final.TARGET
train_x = df_final.drop('TARGET',axis=1)

In [44]:
test_y = df_test.TARGET
test_x = df_test.drop('TARGET',axis=1)

In [45]:
min_max_scaler = preprocessing.MinMaxScaler(feature_range=(0,5))
train_x = min_max_scaler.fit_transform(train_x)
test_x = min_max_scaler.fit_transform(test_x)

In [46]:
from sklearn import linear_model
reg = linear_model.LogisticRegression()
reg.fit(train_x,train_y)
score = reg.score(test_x,test_y)
print('Logistic Regression Accuracy : ',score)

Logistic Regression Accuracy :  0.7845


In [47]:
from sklearn.metrics import classification_report
print(classification_report(test_y,reg.predict(test_x)))

             precision    recall  f1-score   support

          0       0.99      0.78      0.87      1920
          1       0.13      0.79      0.23        80

avg / total       0.95      0.78      0.85      2000



In [48]:
reg2 = linear_model.LogisticRegression(class_weight='balanced')
reg2.fit(train_x,train_y)
score2 = reg2.score(test_x,test_y)
print('Logistic Regression used class weight ' ,score2)
print(classification_report(test_y,reg2.predict(test_x)))

Logistic Regression used class weight  0.807
             precision    recall  f1-score   support

          0       0.99      0.81      0.89      1920
          1       0.14      0.78      0.24        80

avg / total       0.95      0.81      0.86      2000



In [49]:
# fit model no training data
model = XGBClassifier(learning_rate=0.2,max_depth=7,base_score=0.5,max_delta_step=0,n_estimators=100)

pipe = Pipeline([['xg',MinMaxScaler(feature_range=(0,5))],['model',model]])
pipe.fit(train_x,train_y)
print(classification_report(test_y,pipe.predict(test_x)))
print("ACCURACY:",pipe.score(test_x,test_y))

             precision    recall  f1-score   support

          0       0.98      0.88      0.93      1920
          1       0.17      0.56      0.26        80

avg / total       0.95      0.87      0.90      2000

ACCURACY: 0.872
